In [ ]:
!git clone https://github.com/gokhaneraslan/stable-diffusion-3.5-lora-finetuning.git

In [ ]:
!pip install -r /content/stable-diffusion-3.5-lora-finetuning/requirements.txt

In [ ]:
!huggingface-cli login --token your_hugginface_token --add-to-git-credential

In which compute environment are you running?
[0] This machine
[1] AWS (Amazon SageMaker)
-> 0 (This machine)

Which type of machine are you using?
[0] No distributed training
[1] multi-CPU
[2] multi-GPU
...
-> 0 (No distributed training, if you are on a single GPU)

Do you want to run your training on CPU only? [yes/NO]:
-> NO

Do you wish to optimize your script with torch dynamo? [yes/NO]:
-> NO

Do you want to use DeepSpeed? [yes/NO]:
-> NO

What GPU(s) (by id) should be used for training on this machine? (e.g. '0,1,2,3', 'all') [all]:
-> all (or 0)

Would you like to enable numa efficiency? [yes/NO]:
-> NO (This is safe for most systems)

Do you wish to use mixed precision?
[0] no
[1] fp16
[2] bf16
...
-> 2 (bf16, to match our project's config)

In [ ]:
accelerate config

In [ ]:
!pip install -U datasets # To avoid any problems while loading the dataset

In [ ]:
# You can download a dataset from huggingface for testing and try it out.

from datasets import load_dataset
from pathlib import Path
import json

def download_dataset(save_path):
    dataset = load_dataset("moving-j/ghibli-style-100")

    save_path = Path(save_path)
    images_dir = save_path / "images"
    images_dir.mkdir(parents=True, exist_ok=True)

    metadata = []
    for i, item in enumerate(dataset['train']):
        # Save image
        image_path = f"test{i:03d}.png"
        item['image'].save(images_dir / image_path)

        # Prepare metadata
        metadata.append({
            "file_name": image_path,
            "text": item['text']
        })

    # create metadata.jsonl
    with open(save_path / "metadata.jsonl", 'w') as f:
        for entry in metadata:
            f.write(json.dumps(entry) + '\n')


download_dataset("/content/MyImgDataset")

In [ ]:
!accelerate launch /content/stable-diffusion-3.5-lora-finetuning/train.py --config_path /content/stable-diffusion-3.5-lora-finetuning/configs/training_config.yaml

In [ ]:
!python /content/drive/MyDrive/MyImgDataset/lora_fine_tuning/inference.py \
    --base_model_id "stabilityai/stable-diffusion-3.5-medium" \
    --lora_checkpoint_dir "/content/stable-diffusion-3.5-lora-finetuning/sd3_lora_output/final" \
    --prompt "anime style, a girl with blue hair in a magical forest, detailed, beautiful lighting" \
    --steps 28 \
    --guidance_scale 7.0 \
    --resolution 512 \
    --device "cuda" \
    --output_path "/content/anime_style_girl.png"

In [ ]:
from IPython.display import display
from PIL import Image

image = Image.open("/content/gaya_style_girl.png")

display(image)